### Registered EVs and Hybrid Vehicles per PC4

#### from 2017 to 2022

In [44]:
import pandas as pd

# read the the data of registered EVs and Hybrid Vehicles per PC4
data_EV = pd.read_csv(r'path\to\file\Elektrische en waterstofvoertuigen - Postcodes (PC4 incl. Postbussen) van G.csv', sep=';')

In [45]:
ev_ams_total = data_EV.iloc[:, :73] # total number includes all other type of vehicle than just passenger car, ie. truck, van,...
ev_ams_passenger = data_EV.iloc[:, 73:145]
hybird_ams_total = data_EV.iloc[:, 145:217]
hybird_ams_passenger = data_EV.iloc[:, 217:] 

In [46]:
# Rename the columns to a time series
ev_ams_passenger.columns = ev_ams_passenger.columns.str.split('|').str[1] # only retain the time value of the column names
ev_ams_passenger['PC4'] = data_EV['Postcodes (PC4 incl. Postbussen)'] # get the equivalent PC4 level for the observations
ev_ams_passenger = ev_ams_passenger[ev_ams_passenger['PC4'].astype(float) >= 1011] # only retain the active postcodes in Amsterdam which is from 1011
ev_ams_passenger = ev_ams_passenger.set_index('PC4')

In [47]:
# Rename the columns to a time series
hybird_ams_passenger.columns = hybird_ams_passenger.columns.str.split('|').str[1] # only retain the time value of the column names
hybird_ams_passenger['PC4'] = data_EV['Postcodes (PC4 incl. Postbussen)'] # get the equivalent PC4 level for the observations
hybird_ams_passenger = hybird_ams_passenger[hybird_ams_passenger['PC4'].astype(float) >= 1011] # only retain the active postcodes in Amsterdam which is from 1011
hybird_ams_passenger = hybird_ams_passenger.set_index('PC4')

In [48]:
ev_ams_passenger = ev_ams_passenger[['December 2017', 'December 2018','December 2019','December 2020','December 2021','December 2022']] # only include the last month of each year
ev_ams_passenger.columns = ev_ams_passenger.columns.map(lambda x: f"{x}_EV") 

In [49]:
ev_ams_passenger.head()

,December 2017_EV,December 2018_EV,December 2019_EV,December 2020_EV,December 2021_EV,December 2022_EV
PC4,,,,,,
1011,56.0,78.0,136.0,198.0,296.0,367.0
1012,20.0,30.0,48.0,59.0,67.0,79.0
1013,647.0,1316.0,2539.0,3160.0,3483.0,228.0
1014,13.0,28.0,64.0,135.0,252.0,363.0
1015,19.0,27.0,54.0,70.0,106.0,116.0


In [50]:
hybird_ams_passenger = hybird_ams_passenger[['December 2017', 'December 2018','December 2019','December 2020','December 2021','December 2022']] # only include the last month of each year
hybird_ams_passenger.columns = hybird_ams_passenger.columns.map(lambda x: f"{x}_hybrid")

In [51]:
hybird_ams_passenger.head()

,December 2017_hybrid,December 2018_hybrid,December 2019_hybrid,December 2020_hybrid,December 2021_hybrid,December 2022_hybrid
PC4,,,,,,
1011,43.0,39.0,49.0,36.0,69.0,187.0
1012,35.0,35.0,36.0,40.0,3273.0,8136.0
1013,79.0,76.0,79.0,93.0,122.0,154.0
1014,70.0,61.0,56.0,51.0,62.0,82.0
1015,45.0,47.0,59.0,75.0,92.0,109.0


In [372]:
# save the DataFrame to a CSV file
#ev_ams_passenger.to_csv('ev_ams_passenger.csv', index=True)
#hybird_ams_passenger.to_csv('hybird_ams_passenger.csv', index=True)

### Charging locations

#### at the moment

In [37]:
import pandas as pd
# read the the data of charging locations per PC6
data_charger = pd.read_csv(r'path\to\file\output_chargers.csv', sep=',')

In [38]:
# Clean the 'power' column
for index, row in data_charger.iterrows():
    if 'kW' in row['power']:
        data_charger.at[index, 'power'] = row['power'].replace('kW', '')

for index, row in data_charger.iterrows():
    if '2x' in row['power']:
        data_charger.at[index, 'power'] = row['power'].replace('2x', '')


In [35]:
import geopy
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="geoapiExercises")

# Define a function to get the zip code from coordinates
def get_zip_code_from_coords(latitude, longitude):
    point = geopy.Point(latitude, longitude)
    location = geolocator.reverse(point)
    return location.raw['address']['postcode']


In [39]:
for index, row in data_charger.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    zipcode = get_zip_code_from_coords(latitude, longitude)
    data_charger.at[index, 'zipcode'] = zipcode

# Error because of one NaN observation, the rest is accounted for

KeyError: 'postcode'

In [41]:
data_charger.head()

,charger_ID,city,address,provider,latitude,longitude,operator,type,power,zipcode
0,NLALLEGO006110,Amsterdam,Hobbemakade 121,ALLEGO,52.350267,4.886010,NaN,FastCharger,50,1071 XW
1,NLALLEGO006001,Amsterdam,Transformatorweg 28,ALLEGO,52.392331,4.852782,NaN,FastCharger,50,1014 AK
2,NLALLEGO007904,Amsterdam,Europaboulevard 10,ALLEGO,52.333660,4.888636,NaN,FastCharger,50,1083 AD
3,NLALLEGO006000,Amsterdam,Transformatorweg 28,ALLEGO,52.392331,4.852782,NaN,FastCharger,50,1014 AK
4,EVB-DC140100001,Amsterdam,Anthony Fokkerweg 2,EVBOX,52.343528,4.841990,NaN,FastCharger,50,1059 CM


In [52]:
# Get the PC4 and PC6
data_charger['postcode'] = data_charger['zipcode'].str[:4]
data_charger = data_charger.rename(columns={'zipcode':'PC6'})

# Format power correctly
data_charger['power'] = round(data_charger['power'].astype(float))

# Filter the groups with only Amsterdam locations
data_charger = data_charger[data_charger['city'] == 'Amsterdam']

# group the data by postcode
a = data_charger.groupby(['PC6']).agg({'power': 'count'})
b = data_charger.groupby(['PC6']).agg({'power': 'sum'})


In [53]:
# Merge the count of charger and sum of power for each PC6
data_charger_by_postcode = pd.merge(a,b,on='PC6')
data_charger_by_postcode = data_charger_by_postcode.rename(columns={'power_y':'total_capacity',
                                                                    'power_x':'number_of_charger'})
data_charger_by_postcode.index = data_charger_by_postcode.index.str.replace(' ', '')

In [55]:
# Exclude the chargers in the other gemeente around Amsterdam
data_charger_by_postcode_ams = data_charger_by_postcode[(data_charger_by_postcode.index.str[:4].astype(float) < 1110) & (data_charger_by_postcode.index.str[:4].astype(float) >= 1011)]

In [56]:
data_charger_by_postcode_ams.head()

,number_of_charger,total_capacity
PC6,,
1011AB,2,22.0
1011AC,1,11.0
1011BG,1,22.0
1011CA,1,11.0
1011EG,1,11.0


In [ ]:
# save the DataFrame to a CSV file
#data_charger_by_postcode_ams.to_csv('data_charger_by_postcode_ams.csv', index=True)

### Energy Supply per Postcode

#### in 2020 

In [62]:
# read the the data for energy supply
data_energy = pd.read_csv(r'path\to\file\Publicatiefile_Energie_postcode6_2020.csv', sep=';')

# select only the relevant column
data_energy = data_energy[['Postcode6','Gemiddelde_elektriciteitslevering_woningen','Gemiddelde_elektriciteitslevering_bedrijven']]

# rename the columns
data_energy = data_energy.rename(columns={'Gemiddelde_elektriciteitslevering_woningen':'electricity_household',
                                          'Gemiddelde_elektriciteitslevering_bedrijven':'electricity_company',
                                          'Postcode6':'PC6'})

In [64]:
# Clean the data
for index, row in data_energy.iterrows():
    if '.' in row['electricity_household']:
        data_energy.at[index, 'electricity_household'] = row['electricity_household'].replace('.', '0')

for index, row in data_energy.iterrows():
    if '.' in row['electricity_company']:
        data_energy.at[index, 'electricity_company'] = row['electricity_company'].replace('.', '0')


In [68]:
# Change the data type
data_energy['electricity_household'] = data_energy['electricity_household'].astype(int)
data_energy['electricity_company'] = data_energy['electricity_company'].astype(int)

# Filter the postcodes of Amsterdam
data_energy_by_postcode = data_energy[data_energy['PC6'].str[:4].astype(int) < 1110]

# Group the data by postcode
data_energy_by_postcode = data_energy_by_postcode.groupby(['PC6']).agg({'electricity_household': 'sum',
                                                                        'electricity_company':'sum'})

In [70]:
data_energy_by_postcode.head()

,electricity_household,electricity_company
PC6,,
1011AB,0,62110
1011AC,2880,68890
1011AD,0,971750
1011AE,0,0
1011AG,0,32060


In [ ]:
# save the DataFrame to a CSV file
#data_energy_by_postcode.to_csv('data_energy_by_postcode.csv', index=True)

### Demographic

#### Yearly 1998 to 2022

In [80]:
# read the the data of demographic
data_demographic = pd.read_csv(r'path\to\file\Observations.csv', sep=';')

In [81]:
# select only the relevant column
data_demographic = data_demographic[['Leeftijd','Postcode','Perioden','Value']]

# rename the columns
data_demographic = data_demographic.rename(columns={'Leeftijd':'Age',
                                                    'Postcode':'PC4',
                                                    'Perioden':'Year',
                                                    'Value':'Population'})

In [82]:
# Retain only the numeric portion of the year
data_demographic['Year'] = data_demographic['Year'].str[:4].astype(int) 

# Retain only the most recent year
data_demographic = data_demographic[data_demographic['Year'] > 2016] 

# Get the PC4 and filter out the Amsterdam observation
data_demographic['PC4'] = data_demographic['PC4'].str[2:].astype(int)
data_demographic = data_demographic[(data_demographic['PC4'] >= 1000) & (data_demographic['PC4'] <= 1109)]

In [83]:
# Read the data of decoding age
data_demographic_age = pd.read_csv(r'path\to\file\LeeftijdCodes.csv', sep=';')
data_demographic_age = pd.DataFrame({'Identifier':data_demographic_age['Identifier'],
                                            'Title':['Total','0_to_10','0_to_10',
                                                     '10_to_20','10_to_20',
                                                     '20_to_30','20_to_30',
                                                     '30_to_40','30_to_40',
                                                     '40_to_50','40_to_50',
                                                     '50_to_60','50_to_60',
                                                     '60_to_70','60_to_70',
                                                     '70_to_80','70_to_80',
                                                     '80_to_90','80_to_90',
                                                     '90_older','90_older']}) # change the range from 5 to 10 years

In [84]:
# Decoding the age
data_demographic['Age'] = data_demographic['Age'].replace(data_demographic_age.set_index('Identifier')['Title'])

In [85]:
# Set index
data_demographic = data_demographic.set_index('PC4')

In [86]:
data_demographic.head()

,Age,Year,Population
PC4,,,
1011,Total,2017,9645.0
1011,Total,2018,9765.0
1011,Total,2019,9670.0
1011,Total,2020,9705.0
1011,Total,2021,9630.0


##### Population Total

In [87]:
# Retrieve the data of overall population throughout the year per PC4
data_population = data_demographic[data_demographic['Age'] == 'Total'].drop(columns='Age', axis=1)
data_population = data_population.pivot(columns='Year', values='Population')

In [89]:
data_population.head()

Year,2017,2018,2019,2020,2021,2022
PC4,,,,,,
1011,9645.0,9765.0,9670.0,9705.0,9630.0,9760.0
1012,8235.0,8235.0,8420.0,8600.0,8540.0,8800.0
1013,21080.0,21135.0,21335.0,21395.0,21255.0,21940.0
1014,645.0,620.0,1740.0,2180.0,2870.0,3660.0
1015,14810.0,14895.0,14885.0,14865.0,14850.0,14910.0


In [23]:
#data_population.to_csv('data_population.csv', index=True)

##### Population Age

In [90]:
# Dedoding the age group
data_population_age = data_demographic
data_population_age['Age'] = data_population_age['Age'].replace(data_demographic_age.set_index('Identifier')['Title'])

In [92]:
# Calculate total population by PC4, Age and Year
data_population_age = data_population_age.groupby(['PC4','Age','Year']).sum()

# Reshape the data
data_population_age = pd.pivot_table(data_population_age, values='Population', index='PC4', columns=['Year','Age'])
data_population_age.columns = data_population_age.columns.map('{0[0]}_{0[1]}'.format) 

In [94]:
data_population_age.head()

,2017_0_to_10,2017_10_to_20,2017_20_to_30,2017_30_to_40,2017_40_to_50,2017_Total,2018_0_to_10,2018_10_to_20,2018_20_to_30,2018_30_to_40,...,2021_20_to_30,2021_30_to_40,2021_40_to_50,2021_Total,2022_0_to_10,2022_10_to_20,2022_20_to_30,2022_30_to_40,2022_40_to_50,2022_Total
PC4,,,,,,,,,,,,,,,,,,,,,
1011,595.0,515.0,1970.0,1530.0,1240.0,9645.0,585.0,505.0,2060.0,1520.0,...,1975.0,1550.0,1120.0,9630.0,475.0,540.0,2065.0,1645.0,1080.0,9760.0
1012,350.0,265.0,2795.0,1900.0,1040.0,8235.0,335.0,270.0,2775.0,1940.0,...,2880.0,2055.0,1000.0,8540.0,280.0,330.0,3120.0,2045.0,1010.0,8800.0
1013,1950.0,1365.0,3230.0,4040.0,3280.0,21080.0,1935.0,1365.0,3265.0,3950.0,...,3280.0,3960.0,3040.0,21255.0,1600.0,1670.0,3805.0,4000.0,2920.0,21940.0
1014,60.0,15.0,175.0,215.0,85.0,645.0,60.0,20.0,140.0,205.0,...,315.0,770.0,475.0,2870.0,625.0,245.0,395.0,945.0,610.0,3660.0
1015,1010.0,820.0,2645.0,2610.0,2200.0,14810.0,965.0,835.0,2740.0,2575.0,...,2895.0,2600.0,1890.0,14850.0,815.0,925.0,2920.0,2630.0,1825.0,14910.0


In [22]:
#data_population_age.to_csv('data_population_age.csv', index=True)

# Public Transport

In [95]:
# read the the data of tram and metro in Amsterdam
data_public_transport = pd.read_csv(r'path\to\file\TRAMMETRO_PUNTEN_2022.csv', sep=';')

In [96]:
import geopy
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="geoapiExercises")

def get_zip_code_from_coords(latitude, longitude):
    point = geopy.Point(latitude, longitude)
    location = geolocator.reverse(point)
    address = location.raw.get('address')
    if address and 'postcode' in address:
        return address['postcode']
    else:
        return None

In [97]:
for index, row in data_public_transport.iterrows():
    latitude = row['LAT']
    longitude = row['LNG']
    zipcode = get_zip_code_from_coords(latitude, longitude)
    data_public_transport.at[index, 'zipcode'] = zipcode


In [98]:
data_public_transport = data_public_transport[['Naam','Modaliteit','zipcode']] # only keep relevant columns
data_public_transport['zipcode'] = data_public_transport['zipcode'].str.replace(" ", "")

In [99]:
# Filter out the Amsterdam observations
data_public_transport = data_public_transport[data_public_transport['zipcode'].str[:4].astype(float) < 1110]
data_public_transport = data_public_transport.rename(columns={'zipcode':'PC6'}) # rename the PC6 

In [100]:
data_public_transport.head()

,Naam,Modaliteit,PC6
0,Nieuwezijds Kolk,Tram,1012SC
1,Surinameplein / Curaçaostraat,Tram,1058GL
2,Johan Huizingalaan,Tram,1065HZ
3,Prinsengracht,Tram,1017NX
4,Keizergracht,Tram,1017DK


In [101]:
# Group the data by 'zipcode' and 'Modaliteit' columns and get the count of each group
count_df = data_public_transport.groupby(['PC6', 'Modaliteit']).size().reset_index(name='count')

# Pivot the data to create a new dataframe with 'zipcode' as the index, 'Tram' and 'Metro' as columns, and the count as the values
data_public_transport_by_postcode = count_df.pivot(index='PC6', columns='Modaliteit', values='count').fillna(0)

# Rename the columns to 'tram_count' and 'metro_count'
data_public_transport_by_postcode = data_public_transport_by_postcode.rename(columns={'Tram': 'tram_count', 'Metro': 'metro_count'})


In [103]:
data_public_transport_by_postcode.head()

Modaliteit,metro_count,tram_count
PC6,,
1011AC,0.0,1.0
1011MB,1.0,0.0
1011PZ,0.0,1.0
1011RL,1.0,1.0
1012AB,1.0,1.0


In [ ]:
#data_public_transport_by_postcode.to_csv('data_public_transport_by_postcode.csv', index=True)

## Buses

In [104]:
# Read the data of buses in Amsterdam
data_bus = pd.read_excel(r'path\to\file\Bus_Parking_Amsterdam.xlsx', sheet_name='bus')
data_bus['Postcode'] = data_bus['Postcode'].str.replace(" ", "")

In [105]:
# Group the data by 'Postcode' column and get the count of each group
data_bus_by_postcode = data_bus.groupby('Postcode').size().reset_index(name='count')
data_bus_by_postcode = data_bus_by_postcode.rename(columns={'count': 'bus_count',
                                                            'Postcode':'PC6'})

In [106]:
# Set index
data_bus_by_postcode = data_bus_by_postcode.set_index('PC6')

In [107]:
data_bus_by_postcode.head()

,bus_count
PC6,
1012AB,1
1012TK,1
1013DJ,1
1013GM,1
1013KN,1


In [ ]:
#data_bus_by_postcode.to_csv('data_bus_by_postcode.csv', index=True)

## P+R Parking

In [108]:
# Read the data of P+R facilities
data_parking = pd.read_excel(r'path\to\file\Bus_Parking_Amsterdam.xlsx', sheet_name='parking')
data_parking['Postcode'] = data_parking['Postcode'].str.replace(" ", "")
data_parking = data_parking.rename(columns={'Postcode':'PC6'})

In [109]:
# Group the data by 'Postcode' column and get the count of each group
data_parking_by_postcode = data_parking.groupby('PC6').size().reset_index(name='parking_count')

In [110]:
# Set index
data_parking_by_postcode = data_parking_by_postcode.set_index('PC6')

In [111]:
data_parking_by_postcode.head()

,parking_count
PC6,
1022BP,1
1025XB,1
1043DW,1
1055JE,1
1076DE,1


In [ ]:
#data_parking_by_postcode.to_csv('data_parking_by_postcode.csv', index=True)

# Social Economics

In [112]:
# read the data of social economics information
data_social_economics_total = pd.read_excel(r'path\to\file\ses-pc4-2014-2019.xlsx', sheet_name='Tabel 1')

In [113]:
# Filter to get the recent year data and the observations of Amsterdam
data_social_economics = data_social_economics_total[data_social_economics_total['Verslagjaar'] > 2016]
data_social_economics = data_social_economics[data_social_economics['Viercijferige postcode'] < 1110]

In [114]:
data_social_economics.head()

,Verslagjaar,Viercijferige postcode,Aantal huishoudens,Vermogen: 1e tot en met 40e percentielgroep,Vermogen: 41e tot en met 80e percentielgroep,Vermogen: 81e tot en met 100e percentielgroep,Vermogen: Gemiddelde percentielgroep,Gestandaardiseerd inkomen: 1e tot en met 40e percentielgroep,Gestandaardiseerd inkomen: 41e tot en met 80e percentielgroep,Gestandaardiseerd inkomen: 81e tot en met 100e percentielgroep,...,SES-WOA deelscore financiële welvaart: gemiddelde score,SES-WOA deelscore arbeidverleden: gemiddelde score,Ondergrens 95%-interval SES WOA totaalscore,Gemiddelde SES WOA totaalscore,Bovengrens 95%-interval SES WOA totaalscore,Spreiding welvaart,Spreiding arbeidsverleden,Ondergrens 95%-interval van de spreiding in de SES WOA score,Spreiding van de SES WOA score,Bovengrens 95%-interval van de spreiding in de SES WOA score
12115,2017.0,1011.0,6100,46.3,29.7,24,49,45.9,26.4,27.6,...,-0.049,-0.042,0.018,0.021,0.025,0.46,0.372,0.927,0.93,0.934
12116,2017.0,1012.0,5700,58,26.8,15.2,40.4,50,27.9,22.1,...,-0.111,-0.045,-0.039,-0.036,-0.032,0.471,0.374,0.896,0.9,0.904
12117,2017.0,1013.0,12300,51.6,31.8,16.7,44.7,48.2,28.8,23,...,-0.088,-0.063,-0.085,-0.083,-0.081,0.46,0.409,1.02,1.022,1.024
12118,2017.0,1014.0,300,57.4,30.9,11.7,38.6,25.9,30.6,43.5,...,0.063,0.032,0.245,0.259,0.273,0.381,0.335,0.826,0.847,0.868
12119,2017.0,1015.0,9400,45.2,29.2,25.6,50.2,45.8,26.6,27.6,...,-0.043,-0.05,0.016,0.019,0.022,0.46,0.38,0.936,0.938,0.941


#### Total Household

In [115]:
data_household = data_social_economics[['Verslagjaar','Viercijferige postcode','Aantal huishoudens']]
data_household = data_household.rename(columns={'Verslagjaar':'Year',
                                                'Viercijferige postcode': 'PC4',
                                                'Aantal huishoudens':'household_total'})

In [116]:
# Reshape the data to the right format
data_household = data_household.pivot(index='PC4', columns='Year', values='household_total')
data_household.columns = data_household.columns.map(lambda x: f"{x}_household")

In [117]:
data_household.head()

Year,2017.0_household,2018.0_household,2019.0_household
PC4,,,
1011.0,6100,6100,6200
1012.0,5700,5600,5700
1013.0,12300,12300,12300
1014.0,300,300,900
1015.0,9400,9500,9400


In [118]:
#data_household.to_csv('data_household.csv', index=True)

#### Assets

In [121]:
data_asset = data_social_economics[['Verslagjaar','Viercijferige postcode','Vermogen: 1e tot en met 40e percentielgroep','Vermogen: 41e tot en met 80e percentielgroep','Vermogen: 81e tot en met 100e percentielgroep','Vermogen: Gemiddelde percentielgroep']]
data_asset = data_asset.rename(columns={'Verslagjaar':'Year',
                                        'Viercijferige postcode': 'PC4',
                                        'Vermogen: 1e tot en met 40e percentielgroep':'assets_1_to_40_percent',
                                        'Vermogen: 41e tot en met 80e percentielgroep':'assets_41_to_80_percent',
                                        'Vermogen: 81e tot en met 100e percentielgroep':'assets_81_to_100_percent',
                                        'Vermogen: Gemiddelde percentielgroep':'assets_average'})

In [122]:
# Reshape the data to the right format
data_asset_by_postcode = data_asset.pivot(index='PC4', columns='Year', values=['assets_1_to_40_percent','assets_41_to_80_percent','assets_81_to_100_percent','assets_average'])
data_asset_by_postcode.columns = data_asset_by_postcode.columns.map('{0[1]}_{0[0]}'.format) 
data_asset_by_postcode = data_asset_by_postcode.replace(".", "")

In [123]:
data_asset_by_postcode.head()

,2017.0_assets_1_to_40_percent,2018.0_assets_1_to_40_percent,2019.0_assets_1_to_40_percent,2017.0_assets_41_to_80_percent,2018.0_assets_41_to_80_percent,2019.0_assets_41_to_80_percent,2017.0_assets_81_to_100_percent,2018.0_assets_81_to_100_percent,2019.0_assets_81_to_100_percent,2017.0_assets_average,2018.0_assets_average,2019.0_assets_average
PC4,,,,,,,,,,,,
1011.0,46.3,48.3,50.8,29.7,26.6,23,24,25,26.2,49,48.5,47.6
1012.0,58,61.1,63.1,26.8,22.9,20.5,15.2,16,16.4,40.4,39.1,38.2
1013.0,51.6,53,54.7,31.8,27.8,25.2,16.7,19.1,20.1,44.7,44.9,44.2
1014.0,57.4,57.4,46.7,30.9,27.5,27.4,11.7,15.1,25.9,38.6,39.1,49.2
1015.0,45.2,47.7,50.2,29.2,25.3,23.3,25.6,27.1,26.5,50.2,49.6,48.3


In [375]:
#data_asset_by_postcode.to_csv('data_asset_by_postcode.csv', index=True)

#### Income

In [124]:
data_income = data_social_economics[['Verslagjaar','Viercijferige postcode','Gestandaardiseerd inkomen: 1e tot en met 40e percentielgroep','Gestandaardiseerd inkomen: 41e tot en met 80e percentielgroep','Gestandaardiseerd inkomen: 81e tot en met 100e percentielgroep','Gestandaardiseerd inkomen: Gemiddelde percentielgroep']]
data_income = data_income.rename(columns={'Verslagjaar':'Year',
                                        'Viercijferige postcode': 'PC4',
                                        'Gestandaardiseerd inkomen: 1e tot en met 40e percentielgroep':'income_1_to_40_percent',
                                        'Gestandaardiseerd inkomen: 41e tot en met 80e percentielgroep':'income_41_to_80_percent',
                                        'Gestandaardiseerd inkomen: 81e tot en met 100e percentielgroep':'income_81_to_100_percent',
                                        'Gestandaardiseerd inkomen: Gemiddelde percentielgroep':'income_average'})

In [125]:
# Reshape the data to the right format
data_income_by_postcode = data_income.pivot(index='PC4', columns='Year', values=['income_1_to_40_percent','income_41_to_80_percent','income_81_to_100_percent','income_average'])
data_income_by_postcode.columns = data_income_by_postcode.columns.map('{0[1]}_{0[0]}'.format) 
data_income_by_postcode = data_income_by_postcode.replace(".", "")

In [126]:
data_income_by_postcode.head()

,2017.0_income_1_to_40_percent,2018.0_income_1_to_40_percent,2019.0_income_1_to_40_percent,2017.0_income_41_to_80_percent,2018.0_income_41_to_80_percent,2019.0_income_41_to_80_percent,2017.0_income_81_to_100_percent,2018.0_income_81_to_100_percent,2019.0_income_81_to_100_percent,2017.0_income_average,2018.0_income_average,2019.0_income_average
PC4,,,,,,,,,,,,
1011.0,45.9,45.1,45,26.4,27.7,28,27.6,27.2,27.1,49,49.4,49.3
1012.0,50,47.9,47.1,27.9,29.9,29.8,22.1,22.2,23.1,44.3,45.3,46.2
1013.0,48.2,47.6,46.4,28.8,29,29.4,23,23.3,24.2,46.8,47.3,48.1
1014.0,25.9,24.8,24.8,30.6,29.3,22.7,43.5,45.9,52.5,62,63.4,66.6
1015.0,45.8,45.1,44.4,26.6,27.4,27.9,27.6,27.5,27.7,49.2,49.5,50.1


In [377]:
#data_income_by_postcode.to_csv('data_income_by_postcode.csv', index=True)

#### SES-WOA Score

In [127]:
data_ses_woa = data_social_economics[['Verslagjaar','Viercijferige postcode','SES-WOA deelscore financiële welvaart: gemiddelde score','SES-WOA deelscore arbeidverleden: gemiddelde score','Ondergrens 95%-interval SES WOA totaalscore','Gemiddelde SES WOA totaalscore','Bovengrens 95%-interval SES WOA totaalscore','Spreiding welvaart','Spreiding arbeidsverleden','Ondergrens 95%-interval van de spreiding in de SES WOA score','Spreiding van de SES WOA score','Bovengrens 95%-interval van de spreiding in de SES WOA score']]
data_ses_woa = data_ses_woa.rename(columns={'Verslagjaar':'Year',
                                           'Viercijferige postcode': 'PC4'})

In [128]:
# Reshape the data to the right format
data_ses_woa_by_postcode = data_ses_woa.pivot(index='PC4', columns='Year')
data_ses_woa_by_postcode.columns = data_ses_woa_by_postcode.columns.map('{0[1]}_{0[0]}'.format) 
data_ses_woa_by_postcode = data_ses_woa_by_postcode.replace(".", "")

In [129]:
data_ses_woa_by_postcode.head()

,2017.0_SES-WOA deelscore financiële welvaart: gemiddelde score,2018.0_SES-WOA deelscore financiële welvaart: gemiddelde score,2019.0_SES-WOA deelscore financiële welvaart: gemiddelde score,2017.0_SES-WOA deelscore arbeidverleden: gemiddelde score,2018.0_SES-WOA deelscore arbeidverleden: gemiddelde score,2019.0_SES-WOA deelscore arbeidverleden: gemiddelde score,2017.0_Ondergrens 95%-interval SES WOA totaalscore,2018.0_Ondergrens 95%-interval SES WOA totaalscore,2019.0_Ondergrens 95%-interval SES WOA totaalscore,2017.0_Gemiddelde SES WOA totaalscore,...,2019.0_Spreiding arbeidsverleden,2017.0_Ondergrens 95%-interval van de spreiding in de SES WOA score,2018.0_Ondergrens 95%-interval van de spreiding in de SES WOA score,2019.0_Ondergrens 95%-interval van de spreiding in de SES WOA score,2017.0_Spreiding van de SES WOA score,2018.0_Spreiding van de SES WOA score,2019.0_Spreiding van de SES WOA score,2017.0_Bovengrens 95%-interval van de spreiding in de SES WOA score,2018.0_Bovengrens 95%-interval van de spreiding in de SES WOA score,2019.0_Bovengrens 95%-interval van de spreiding in de SES WOA score
PC4,,,,,,,,,,,,,,,,,,,,,
1011.0,-0.049,-0.043,-0.043,-0.042,-0.045,-0.044,0.018,0.02,0.031,0.021,...,0.374,0.927,0.921,0.915,0.93,0.925,0.919,0.934,0.929,0.923
1012.0,-0.111,-0.1,-0.09,-0.045,-0.037,-0.025,-0.039,-0.023,0.006,-0.036,...,0.362,0.896,0.881,0.865,0.9,0.887,0.868,0.904,0.892,0.872
1013.0,-0.088,-0.076,-0.066,-0.063,-0.057,-0.046,-0.085,-0.066,-0.035,-0.083,...,0.394,1.02,1.014,0.994,1.022,1.016,0.997,1.024,1.019,1
1014.0,0.063,0.07,0.102,0.032,0.056,0.046,0.245,0.295,0.313,0.259,...,0.328,0.826,0.792,0.866,0.847,0.803,0.876,0.868,0.814,0.886
1015.0,-0.043,-0.038,-0.032,-0.05,-0.052,-0.044,0.016,0.018,0.035,0.019,...,0.376,0.936,0.937,0.921,0.938,0.94,0.924,0.941,0.942,0.926


In [379]:
#data_ses_woa_by_postcode.to_csv('data_ses_woa_by_postcode.csv', index=True)

# Land Use

In [137]:
# Read the data of land use in Amsterdam
data_land_use = pd.read_excel(r'path\to\file\2020_buurten_4_01_1ded2c0cae.xlsx', sheet_name='2020_buurten_41')
data_land_use = data_land_use.dropna()

# Filter to get the name of the neighborhood (buurt)
data_land_use['buurt'] = data_land_use['buurt'].apply(lambda x: x[5:])

In [139]:
data_land_use.head()

,buurt,verkeersterrein,bebouwd terrein,semi-bebouwd terrein,recreatieterrein,agrarisch terrein,bos en natuurlijk gebied,binnenwater,totaal
0,Kop Zeedijk,0.8236,5.1596,0.0000,0.0,0.0,0.0,2.0607,8.044
1,Oude Kerk e.o.,0.4141,8.4336,0.0213,0.0,0.0,0.0,0.3902,9.2593
2,Burgwallen Oost,0.0000,6.2335,0.0000,0.0,0.0,0.0,1.0152,7.2486
3,Nes e.o.,0.2756,4.2384,0.8811,0.0,0.0,0.0,0.3809,5.7761
4,BG-terrein e.o.,0.0000,8.4299,0.0000,0.0,0.0,0.0,1.7091,10.139


In [140]:
# Read the data of neighborhood name and code
data_neighborhood = pd.read_csv(r'path\to\file\brt2022.csv', sep=';')
data_neighborhood = data_neighborhood[data_neighborhood['GM_NAAM'] == 'Amsterdam']
data_neighborhood = data_neighborhood.rename(columns={'buurtcode2022':'Buurt2022'})

In [141]:
data_neighborhood

,Buurt2022,buurtnaam2022,GM_2022,GM2022,GM_NAAM,WK_2022,WK2022,WK_NAAM
4221,3630000,Kop Zeedijk,GM0363,363,Amsterdam,WK036300,36300,Burgwallen-Oude Zijde
4222,3630001,Oude Kerk e.o.,GM0363,363,Amsterdam,WK036300,36300,Burgwallen-Oude Zijde
4223,3630002,Burgwallen Oost,GM0363,363,Amsterdam,WK036300,36300,Burgwallen-Oude Zijde
4224,3630003,Nes e.o.,GM0363,363,Amsterdam,WK036300,36300,Burgwallen-Oude Zijde
4225,3630004,BG-terrein e.o.,GM0363,363,Amsterdam,WK036300,36300,Burgwallen-Oude Zijde
...,...,...,...,...,...,...,...,...
4697,3639703,Gein Zuidoost,GM0363,363,Amsterdam,WK036397,36397,Gein
4698,3639800,Dorp Driemond,GM0363,363,Amsterdam,WK036398,36398,Driemond
4699,3639801,Landelijk gebied Driemond,GM0363,363,Amsterdam,WK036398,36398,Driemond
4700,3639997,Groot binnenwater,GM0363,363,Amsterdam,WK036399,36399,Groot water


In [142]:
# Read the neighborhood code and its equivalent PC6
data_neighborhood_PC6 = pd.read_csv(r'path\to\file\pc6hnr20220801_gwb.csv', sep=';')
data_neighborhood_PC6 = data_neighborhood_PC6[data_neighborhood_PC6['Gemeente2022'] == 363].drop_duplicates()

In [143]:
data_neighborhood_PC6.head()

,PC6,Huisnummer,Buurt2022,Wijk2022,Gemeente2022
0,1011AB,99,3630400,36304,363
1,1011ER,10,3630403,36304,363
2,1011TD,188,3630402,36304,363
3,1011TN,6,3630402,36304,363
4,1011TN,8,3630402,36304,363


In [145]:
# Merge the datasets to get the match between the PC6 and neighborhood name
data_neighborhood_PC6_merge = data_neighborhood_PC6.merge(data_neighborhood, on='Buurt2022', how='right')
data_neighborhood_PC6_merge = data_neighborhood_PC6_merge[['PC6','Buurt2022','buurtnaam2022']]
data_neighborhood_PC6_merge = data_neighborhood_PC6_merge.drop_duplicates()
data_neighborhood_PC6_merge = data_neighborhood_PC6_merge[['PC6','buurtnaam2022']]
data_neighborhood_PC6_merge = data_neighborhood_PC6_merge.rename(columns={'buurtnaam2022':'buurt'})

In [147]:
data_neighborhood_PC6_merge.head()

,PC6,buurt
0,1012AC,Kop Zeedijk
8,1012AD,Kop Zeedijk
11,1012AE,Kop Zeedijk
32,1012AG,Kop Zeedijk
41,1012AH,Kop Zeedijk


## Merge Land Use

In [148]:
# Merge the datasets to get the match between the PC6 and land use
data_land_use_by_postcode = data_land_use.merge(data_neighborhood_PC6_merge, on='buurt')
data_land_use_by_postcode = data_land_use_by_postcode[['PC6','verkeersterrein','bebouwd terrein','semi-bebouwd terrein','recreatieterrein','agrarisch terrein','bos en natuurlijk gebied','binnenwater','totaal']]
data_land_use_by_postcode = data_land_use_by_postcode.rename(columns={'verkeersterrein':'traffic_area',
                                                                      'bebouwd terrein':'built_up_area',
                                                                      'semi-bebouwd terrein':'semi_built_up_area',
                                                                      'recreatieterrein':'recreational_area',
                                                                      'agrarisch terrein':'agrarian_area',
                                                                      'bos en natuurlijk gebied':'forest_nature_are',
                                                                      'binnenwater':'water_body_area',
                                                                      'totaal':'total_area'})

In [149]:
# Get the average land use of each PC6
data_land_use_by_postcode = data_land_use_by_postcode.groupby('PC6').mean()

In [150]:
data_land_use_by_postcode.head()

,traffic_area,built_up_area,semi_built_up_area,recreational_area,agrarian_area,forest_nature_are,water_body_area
PC6,,,,,,,
1011AB,5.5393,5.3554,1.4589,0.0,0.0,0.0,11.9398
1011AC,5.5393,5.3554,1.4589,0.0,0.0,0.0,11.9398
1011AD,5.5393,5.3554,1.4589,0.0,0.0,0.0,11.9398
1011AE,5.5393,5.3554,1.4589,0.0,0.0,0.0,11.9398
1011AG,0.4505,4.8640,0.0000,0.0,0.0,0.0,2.4582


In [151]:
#data_land_use_by_postcode.to_csv('data_land_use_by_postcode.csv', index=True)

# List of PC6

In [153]:
# Read the data of full list of PC6 in Amsterdam
data_PC6_full = pd.read_excel(r'path\to\file\pc6_2022_v1.xlsx')
data_PC6 = data_PC6_full[data_PC6_full['Postcode-6'].str[:4].astype(float) < 1110]
data_PC6 = data_PC6.rename(columns={'Postcode-6':'PC6'})

In [154]:
data_PC6.head()

,PC6
0,1011AB
1,1011AC
2,1011AG
3,1011AH
4,1011AJ


In [155]:
#data_PC6.to_csv('data_PC6.csv', index=True)